In [1]:
import numpy as np

In [2]:
# Book's example
A = 'ABCBDAB'
B = 'BDCABA'

# Wikipedia's example (https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm)
X = 'GATTACA'
Y = 'GCATGCU'

# Another test taken from 'https://webdocs.cs.ualberta.ca/~games/alignment/'
S1 = 'GTGACTGCGATAAGCTTAGATCCTCTTAAAAT'
S2 = 'GAGGGAGACATGCGATACAAGGGATCCTTGTAGATCTGCGTCTTTAA'

In [3]:
## Complexity = O(n*2^n), where n = len(S)
def find_all_subsequences(S):
    l = []
    for e in range (1,2**len(S)):
        x = ''
        b = format(e, '#0' + str(len(S)+2) + 'b')
        for i in range(2,len(b)):
            if b[i] == '1':
                x += S[i-2]
        l.append(x)
    return l
# testing the function
print(find_all_subsequences('ABC'))

['C', 'B', 'BC', 'A', 'AC', 'AB', 'ABC']


In [4]:
## Complexity = O(n*2^n) + O(2^n*n) = O(n*2^n), where n = len(S)
def exponential_LCS(A,B):
    if len(A)>len(B):
        s = find_all_subsequences(B) 
        S = A
    else:
        s = find_all_subsequences(A)
        S = B
    lcm = ''
    for elem in s:
        if is_subsequence(elem,S):
            if len(lcm)<len(elem):
                lcm = elem
    return len(lcm)

def is_subsequence(str1,str2): ## O(len(n)), n = len(str2)
    m = len(str1)
    n = len(str2)
    if m==0:
        return True
    if n==0:
        return False
    if str1[m-1]==str2[n-1]:
        return is_subsequence(str1[0:m-1],str2[0:n-1])
    else:
        return is_subsequence(str1,str2[0:n-1])
# testing the function
print(exponential_LCS(A,B))
print(exponential_LCS(X,Y))

4
4


In [5]:
def naive_recursive_LCS(A,B):
    if len(A)==0 or len(B)==0:
        return 0
    elif len(A)>0 and len(B)>0:
        if A[len(A)-1] == B[len(B)-1]:
            return 1 + naive_recursive_LCS(A[0:len(A)-1],B[0:len(B)-1])
        else:
            return max(naive_recursive_LCS(A,B[0:len(B)-1]),naive_recursive_LCS(A[0:len(A)-1],B))
# testing the function
print(naive_recursive_LCS(A,B))
print(naive_recursive_LCS(X,Y))

4
4


In [6]:
def recursive_LCS_memoized(A,B):
    m = len(A)
    n = len(B)
    l = -np.ones((m+1,n+1),dtype=int)
    l[0,1:] = np.zeros(n,dtype=int)
    l[:,0] = np.zeros(m+1,dtype=int)
    return LCS_aux(A,B,l)
    
def LCS_aux(A,B,l):
    if l[len(A)][len(B)]!=-1:
        return l[len(A)][len(B)]
    else:
        if A[len(A)-1] == B[len(B)-1]:
            res = 1 + LCS_aux(A[0:len(A)-1],B[0:len(B)-1],l)
            l[len(A)][len(B)] = res
        else:
            a = LCS_aux(A,B[0:len(B)-1],l)
            b = LCS_aux(A[0:len(A)-1],B,l)
            if a>b:
                res = a
                l[len(A)][len(B)-1] = res
            else:
                res = b
                l[len(A)-1][len(B)] = res
        return res
# testing the function
print(recursive_LCS_memoized(A,B))
print(recursive_LCS_memoized(X,Y))
print(recursive_LCS_memoized(S1,S2))

4
4
28


In [7]:
def bottom_up_LCS_memoized(A,B):
    m = len(A)
    n = len(B)
    direc = np.empty((m+1,n+1),dtype=np.str)
    p = np.empty((m+1,n+1),dtype=np.str)
    l = np.zeros((m+1,n+1),dtype=int)
    for i in range(1,m+1):
        for j in range(1,n+1):
            if A[i-1]==B[j-1]:
                l[i][j] = l[i-1][j-1] + 1
                p[i][j] = "L" # L stands for Up and left
                direc[i][j] = '\u2196'
            elif l[i-1][j]<l[i][j-1]:
                l[i][j] = l[i][j-1] 
                p[i][j] = "l"  # l stands for left
                direc[i][j] = '\u2190'
            elif l[i-1][j]>=l[i][j-1]:
                l[i][j] = l[i-1][j] 
                p[i][j] = "u"  # u stands for Up
                direc[i][j] = '\u2191'
    return (l,p,direc)

In [8]:
l,p,d = bottom_up_LCS_memoized(A,B)

In [9]:
print(l)

[[0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1]
 [0 1 1 1 1 2 2]
 [0 1 1 2 2 2 2]
 [0 1 1 2 2 3 3]
 [0 1 2 2 2 3 3]
 [0 1 2 2 3 3 4]
 [0 1 2 2 3 4 4]]


In [10]:
print(d)

[['' '' '' '' '' '' '']
 ['' '↑' '↑' '↑' '↖' '←' '↖']
 ['' '↖' '←' '←' '↑' '↖' '←']
 ['' '↑' '↑' '↖' '←' '↑' '↑']
 ['' '↖' '↑' '↑' '↑' '↖' '←']
 ['' '↑' '↖' '↑' '↑' '↑' '↑']
 ['' '↑' '↑' '↑' '↖' '↑' '↖']
 ['' '↖' '↑' '↑' '↑' '↖' '↑']]


In [11]:
def print_LCS(p,A,i,j):
    if i==0 or j==0:
        return
    if p[i][j] == 'L':
        print_LCS(p,A,i-1,j-1)
        print(A[i-1],end='')
    elif p[i][j] == 'u':
        print_LCS(p,A,i-1,j)
    else:
        print_LCS(p,A,i,j-1)

In [12]:
print_LCS(p,A,len(A),len(B))

BCBA

In [13]:
l,p,_ = bottom_up_LCS_memoized(X,Y)
print_LCS(p,X,len(X),len(Y))

GATC

In [14]:
l,p,_ = bottom_up_LCS_memoized(S1,S2)
print_LCS(p,S1,len(S1),len(S2))

GGACTGCGATAAGCTTAGATCCTCTTAA

In [15]:
l1 = []
l2 = []
def print_best_alignment(p,A,B,i,j):
    if i==0 or j==0:
        if i==0 and j==0:
            pass
        elif i==0:
            l2.append(B[j-1])
            l1.append('-')
        elif j==0:
            l1.append(A[i-1])
            l2.append('-')
        return
    if p[i][j] == 'L':
        print_best_alignment(p,A,B,i-1,j-1)
        l1.append(A[i-1])
        l2.append(B[j-1])
    elif p[i][j] == 'u':
        print_best_alignment(p,A,B,i-1,j)
        l1.append(A[i-1])
        l2.append('-')
    else:
        print_best_alignment(p,A,B,i,j-1)
        l1.append('-')
        l2.append(B[j-1])

In [16]:
print_best_alignment(p,S1,S2,len(S1),len(S2))
str1 = ''.join(l1)
str2 = ''.join(l2)
print(str1)
print(str2)
l1=[]
l2=[]

-G-TGAC-TGCGAT--AA--G---C-T-TAGATC--C-TC-TTAAAAT
GGA-GACATGCGATACAAGGGATCCTTGTAGATCTGCGTCTTT--AA-


In [17]:
def bottom_up_LCS_memoized_with_scoring(A,B,match=1,mismatch=-1,gap=-1):
    m = len(A)
    n = len(B)
    direc = np.empty((m+1,n+1),dtype=np.str)
    p = np.empty((m+1,n+1),dtype=np.str)
    l = np.zeros((m+1,n+1),dtype=int)
    l[0,1:] = np.array([gap*i for i in range(1,n+1)],dtype=int)
    l[:,0] = np.array([gap*i for i in range(m+1)],dtype=int)
    for i in range(1,m+1):
        for j in range(1,n+1):
            if A[i-1]==B[j-1]:
                l[i][j] = l[i-1][j-1] + match
                p[i][j] = 'L'  
                direc[i][j] = '\u2196'
            else:
                if l[i-1][j-1] + mismatch > max(l[i][j-1] + gap,l[i-1][j] + gap):
                    l[i][j] = l[i-1][j-1] + mismatch
                    p[i][j] = 'L'
                    direc[i][j] = '\u2196'
                else:
                    if l[i][j-1] > l[i-1][j]:
                        l[i][j] = l[i][j-1] + gap
                        p[i][j] = 'l'
                        direc[i][j] = '\u2190'
                        
                    else:
                        l[i][j] = l[i-1][j] + gap
                        p[i][j] = 'u' 
                        direc[i][j] = '\u2191'
    return (l,p,direc)

In [18]:
l,p,d = bottom_up_LCS_memoized_with_scoring(X,Y)

In [19]:
# check wikipedia "https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm" to see that we got right!
print(l)

[[ 0 -1 -2 -3 -4 -5 -6 -7]
 [-1  1  0 -1 -2 -3 -4 -5]
 [-2  0  0  1  0 -1 -2 -3]
 [-3 -1 -1  0  2  1  0 -1]
 [-4 -2 -2 -1  1  1  0 -1]
 [-5 -3 -3 -1  0  0  0 -1]
 [-6 -4 -2 -2 -1 -1  1  0]
 [-7 -5 -3 -1 -2 -2  0  0]]


In [20]:
print(d)

[['' '' '' '' '' '' '' '']
 ['' '↖' '←' '←' '←' '↖' '←' '←']
 ['' '↑' '↖' '↖' '←' '←' '←' '←']
 ['' '↑' '↑' '↑' '↖' '←' '←' '←']
 ['' '↑' '↑' '↑' '↖' '↖' '←' '←']
 ['' '↑' '↑' '↖' '↑' '↑' '↖' '←']
 ['' '↑' '↖' '↑' '↑' '↑' '↖' '←']
 ['' '↑' '↑' '↖' '↑' '↑' '↑' '↖']]


In [21]:
l,p,_ = bottom_up_LCS_memoized_with_scoring("TCATA","TCCTA",match=5,mismatch=-1,gap=-2)

In [22]:
# Check this "https://www.youtube.com/watch?v=aD4Cc4L3qW0" video's example to see that we got right 
print(l)

[[  0  -2  -4  -6  -8 -10]
 [ -2   5   3   1  -1  -3]
 [ -4   3  10   8   6   4]
 [ -6   1   8   9   7  11]
 [ -8  -1   6   7  14  12]
 [-10  -3   4   5  12  19]]


In [23]:
l,p,_ = bottom_up_LCS_memoized_with_scoring(A,B,match=1,mismatch=0,gap=0) # this is equiavlent to just finding LCS :)

In [24]:
print(l)

[[0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1]
 [0 1 1 1 1 2 2]
 [0 1 1 2 2 2 2]
 [0 1 1 2 2 3 3]
 [0 1 2 2 2 3 3]
 [0 1 2 2 3 3 4]
 [0 1 2 2 3 4 4]]


In [26]:
l,p,d = bottom_up_LCS_memoized_with_scoring(S1,S2)

In [27]:
print(l)

[[  0  -1  -2 ..., -45 -46 -47]
 [ -1   1   0 ..., -43 -44 -45]
 [ -2   0   0 ..., -41 -42 -43]
 ..., 
 [-30 -28 -26 ...,   5   7   9]
 [-31 -29 -27 ...,   4   6   8]
 [-32 -30 -28 ...,   5   5   7]]


In [28]:
print(d)

[['' '' '' ..., '' '' '']
 ['' '↖' '←' ..., '←' '←' '←']
 ['' '↑' '↖' ..., '↖' '←' '←']
 ..., 
 ['' '↑' '↖' ..., '↑' '↖' '↖']
 ['' '↑' '↖' ..., '↑' '↖' '↖']
 ['' '↑' '↑' ..., '↖' '↑' '↑']]


In [29]:
# different alignment in this case for these two sequences as now mismatch=gap=-1 and not 0
print_best_alignment(p,S1,S2,len(S1),len(S2))
str1 = ''.join(l1)
str2 = ''.join(l2)
print(str1)
print(str2)
l1=[]
l2=[]

GT----GAC-TGCGAT--AA--G---C-T-TAGATC--C-TCTTAAAAT
GAGGGAGACATGCGATACAAGGGATCCTTGTAGATCTGCGTCTTT-AA-
